# Project forecasting

## Imports

In [20]:
# utils
from utils.market_share import *

from typing import Dict, List, Any
import pickle
from tqdm import tqdm 


Informations about the data [here](https://transp-or.epfl.ch/documents/technicalReports/CS_LPMC.pdf).

## Functions

## Goal

1. Report the market shares predicted by Modelpref for each scenario.
Do they match your expectations? Compare those with the original market shares. [1 point]

## Scenario 1

An additional charge of £1.5 to the car users

## Scenario 2

A decrease of the public transport cost by 20%.

In [22]:
# create a dict best practice 
model_market_share: Dict[str, List] = {
    "Model 4 scenario 2" : [V_3, results_nested]# do it by hand very sketchy part !!!!
}
 
file_name = 'market_shares_forecasting.pickle'
try:
    with open(file_name, 'rb') as f:
        all_market_shares = pickle.load(f)
        print(f'Market shares read from {file_name}')
except FileNotFoundError:
    
    all_market_shares: Dict[str, Any] = {
        model: market_share(utilities, results) 
        for model, (utilities, results) in tqdm(model_market_share.items())
    }
    # Save the calculated market shares to the pickle file
    with open(file_name, 'ab') as f:
        pickle.dump(all_market_shares, f)
    print(f'Market shares calculated and saved in {file_name}')

Market shares read from market_shares_forecasting.pickle


In [28]:
print(all_market_shares.keys())
objects = []
with open(file_name, 'rb') as f:
    while True:
        try:
            obj = pickle.load(f)
            objects.append(obj)
        except EOFError:  # End of file
            break

for obj in objects:  # Loop through all the loaded dictionaries
        print(obj)

dict_keys(['Model 4'])
{'Model 4': {'Walking': IndicatorTuple(value=np.float64(0.16926692227752185), lower=np.float64(0.14472646767660818), upper=np.float64(0.1846492787121359)), 'Cycling': IndicatorTuple(value=np.float64(0.02919112349750728), lower=np.float64(0.01925433359908701), upper=np.float64(0.03618786929567122)), 'Public transportation': IndicatorTuple(value=np.float64(0.361891207822671), lower=np.float64(0.31277417682127334), upper=np.float64(0.48065631237001755)), 'Car': IndicatorTuple(value=np.float64(0.43965074640229995), lower=np.float64(0.35698967705633305), upper=np.float64(0.49994423410794553))}}
{'Model 4 scenario 1': {'Walking': IndicatorTuple(value=np.float64(0.16926692227752185), lower=np.float64(0.150897229493781), upper=np.float64(0.18794663660374886)), 'Cycling': IndicatorTuple(value=np.float64(0.02919112349750728), lower=np.float64(0.021192847392588827), upper=np.float64(0.036235973548737324)), 'Public transportation': IndicatorTuple(value=np.float64(0.361891207